## Routing compared

In [1]:
import sys
!{sys.executable} -m pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 81.0 MB/s eta 0:00:00:00:0100:01


In [2]:
import arcgis
from arcgis.gis import GIS
import pandas
import geopandas
import json
import datetime

#### arc services via current sign-in

In [3]:
gis = GIS("home")

/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:676: UserWarning: You are logged on as jbousqui_EPA with an administrator role, proceed with caution.
  warnings.warn(


In [4]:
# Not used here
def gdf_to_featJSON(gdf):
    """Convert geodataframe to json in esri expected format"""
    temp_gdf = gdf.copy().to_crs(4326)
    feats = json.loads(temp_gdf['geometry'].to_json())['features']
    #return {'features':[{'id':item['id'], 'geometry':item['geometry']['coordinates']} for item in feats]}
    #return {'features':[{'geometry':item['geometry']['coordinates']} for item in feats]}
    return {'features':[{'geometry':{"x":item['geometry']['coordinates'][0], 
                                     "y":item['geometry']['coordinates'][1]}} for item in feats]}

 For ESRI service

In [5]:
def get_od_matrix(origins_json, destinations_json, mode='Drive', impedance='TravelTime'):
    """ Requires license but uses 0.1 credits"""
    # Setup
    #origins_json = gdf_to_featJSON(origin_gdf)  # Set json
    #destinations_json = gdf_to_featJSON(destination_gdf)  # Set json
    gis = GIS("home") # set object
    # Service
    od_cost_matrix = arcgis.network.ODCostMatrixLayer(gis.properties.helperServices.odCostMatrix.url, gis=gis)
    # Solve
    return od_cost_matrix.solve_od_cost_matrix(origins=origins_json, destinations=destinations_json)

In [6]:
def get_simple_routes(origins_json, destinations_json, mode='Drive', impedance='TravelTime'):
    """
    If you have an ArcGIS Developer account, simple routes are included with the monthly free tier of access to location services
    """
    # Setup
    #origins_json = gdf_to_featJSON(origin_gdf)  # Set json
    #destinations_json = gdf_to_featJSON(destination_gdf)  # Set json
    gis = GIS("home") # set object
    # Service
    od_route = arcgis.network.RouteLayer(gis.properties.helperServices.route.url, gis=gis)
    # Solve
    results = {}
    for i, origin in enumerate(origins_json['features']):
        results[str(i+1)] = {}
        for j, dest in enumerate(destinations_json['features']):
            stops = "{"+ "'features':[{},{}]".format(origin, dest) + "}"
            result = od_route.solve(stops=stops, return_directions=False)
            results[str(i+1)][str(j+1)] = result
    return results

In [7]:
def od_matrix_to_df(result):
    """parse get_od_matrix() result to expected dataFrame"""
    # number of destinations
    dest_ids = list(result['odCostMatrix']['1'].keys())  #TODO: always '1'?
    # TODO: dict keys are not sorted - will numbers always be after letters?
    orig_ids = list(result['odCostMatrix'].keys())[1:]
    cols = result['odCostMatrix']['costAttributeNames']
    origins_list = orig_ids * len(dest_ids)
    origins_list.sort()
    # Structure data for df
    data = {'Origin': origins_list,
            'Destination': dest_ids * len(orig_ids),
            cols[0]: [],
            cols[1]: [],
            cols[2]: [],
           }
    # Add results to data
    for i, orig in enumerate(data['Origin']):
        dest = data['Destination'][i]
        row = result['odCostMatrix'][orig][dest]
        data[cols[0]].append(row[0])
        data[cols[1]].append(row[1])
        data[cols[2]].append(row[2])
    df = pandas.DataFrame(data)
    # Convert min to datetime hr:min:sec
    df['TravelTime'] = [int(x*60) for x in df['TravelTime']]
    df['Travel_Time'] = [str(datetime.timedelta(seconds=sec)) for sec in df['TravelTime']]
    df['Distance (m)'] = [int(x*1000) for x in df['Kilometers']]
    return df

In [8]:
def simple_routes_to_df(routes):
    """parse get_simple_routes() result to expected dataFrame"""
    dest_ids = list(routes['1'].keys())
    orig_ids = list(routes.keys())
    origins_list = orig_ids * len(dest_ids)
    origins_list.sort()
    # Structure data for df
    data = {'Origin': origins_list,
            'Destination': dest_ids * len(orig_ids),
            'TravelTime': [],
            'Miles': [],
            'Kilometers': [],
           }
    # Add results to data
    for i, orig in enumerate(data['Origin']):
        dest = data['Destination'][i]
        row = routes[orig][dest]['routes']['features'][0]['attributes']
        data['TravelTime'].append(row['Total_TravelTime'])
        data['Miles'].append(row['Total_Miles'])
        data['Kilometers'].append(row['Total_Kilometers'])
    # Fix up df
    df = pandas.DataFrame(data)
    # Convert min to datetime hr:min:sec
    df['TravelTime'] = [int(x*60) for x in df['TravelTime']]
    df['Travel_Time'] = [str(datetime.timedelta(seconds=sec)) for sec in df['TravelTime']]
    df['Distance (m)'] = [int(x*1000) for x in df['Kilometers']]
    # geometry for route
    routes_geom = routes['1']['1']['routes']['features'][0]['geometry']['paths']

    return df, routes_geom

#### Run on Origins and Destinations

In [ ]:
# User variables
#mode = 'walk'  # 'walk' or 'drive' (default)
#origins_sub = '/arcgis/home/All_Parcels_Tracts_within_Centroids2_1222022.shp'
#destinations_sub = '/arcgis/home/ParksandRec_Tracts_within_12232022.shp'

In [ ]:
origin_featJSON = {'features': [{'geometry': {'x': -87.31819356767737, 'y': 30.488993422614293}},
                                {'geometry': {'x': -87.26166117405631, 'y': 30.513611625200657}},
                                {'geometry': {'x': -87.37984132145506, 'y': 30.411708283704982}},
                                {'geometry': {'x': -87.19751815819674, 'y': 30.5040324196343}},
                                {'geometry': {'x': -87.21971277170069, 'y': 30.440053896777236}},
                                {'geometry': {'x': -87.28119169365233, 'y': 30.449959249077125}},
                                {'geometry': {'x': -87.16257135405533, 'y': 30.49383907594538}},
                                {'geometry': {'x': -87.3239797312026, 'y': 30.44056463997631}},
                                {'geometry': {'x': -87.18561540668905, 'y': 30.486258856493453}},
                                {'geometry': {'x': -87.16674718270922, 'y': 30.482188418937426}}]}

In [ ]:
destination_featJSON = {'features': [{'geometry': {'x': -87.21666449027637, 'y': 30.41992587737401}},
                                     {'geometry': {'x': -87.19691618285101, 'y': 30.46798395108601}},
                                     {'geometry': {'x': -87.1781172119541, 'y': 30.43670810686008}},
                                     {'geometry': {'x': -87.3361320174561, 'y': 30.361806137249978}},
                                     {'geometry': {'x': -87.05178162220702, 'y': 30.393756844497236}},
                                     {'geometry': {'x': -87.1347160696414, 'y': 30.33329818318083}},
                                     {'geometry': {'x': -87.20860990703493, 'y': 30.432267491111418}},
                                     {'geometry': {'x': -87.195239318877, 'y': 30.421667603641467}},
                                     {'geometry': {'x': -87.20962300896456, 'y': 30.408739586863398}},
                                     {'geometry': {'x': -87.21296636699766, 'y': 30.402363005348704}}]}

##### OD Service

In [ ]:
# run subset using esri OD service
result_od = get_od_matrix(origin_featJSON, destination_featJSON)

In [ ]:
df_od = od_matrix_to_df(result_od)

##### Route Service

In [ ]:
# run subset using esri route service
result_routes = get_simple_routes(origin_featJSON, destination_featJSON)

In [ ]:
df_routes, routes_esri = simple_routes_to_df(result_routes)

#### Comparisons

##### df_od == df_routes

In [ ]:
df_od == df_routes

In [ ]:
# Compare travel time
pct_esri_time = ((df_od['TravelTime'] - df_routes['TravelTime'])/df_od['TravelTime'])*100
#3 x-axis is index, y-axis is percent disagreement
pct_esri_time.plot()

In [ ]:
pct_esri_time.where(pct_esri_time>0.0).dropna()

In [ ]:
df_od.iloc[6]

In [ ]:
df_routes.iloc[6]

Focus on first bad result

In [9]:
origin_1 = {'features': [{'geometry': {'x': -87.31819356767737, 'y': 30.488993422614293}}]}

In [10]:
destination_7 = {'features': [{'geometry': {'x': -87.20860990703493, 'y': 30.432267491111418}}]}

In [11]:
result_od = get_od_matrix(origin_1, destination_7)

/opt/conda/lib/python3.9/site-packages/arcgis/gis/__init__.py:676: UserWarning: You are logged on as jbousqui_EPA with an administrator role, proceed with caution.
  warnings.warn(


In [12]:
result_od

{'requestID': '4335e5f8-7c87-4317-a4e4-36ba18a6e0f5',
 'odCostMatrix': {'costAttributeNames': ['TravelTime', 'Miles', 'Kilometers'],
  '1': {'1': [18.617999042074185, 9.867147252389017, 15.87944907045771]}},
 'messages': []}

In [13]:
#result_routes = get_simple_routes(origin_1, destination_7)

In [19]:
for i, origin in enumerate(origin_1['features']):
    for j, dest in enumerate(destination_7['features']):
        stops = "{"+ "'features':[{},{}]".format(origin, dest) + "}"
stops

"{'features':[{'geometry': {'x': -87.31819356767737, 'y': 30.488993422614293}},{'geometry': {'x': -87.20860990703493, 'y': 30.432267491111418}}]}"

In [20]:
od_route = arcgis.network.RouteLayer(gis.properties.helperServices.route.url, gis=gis)
result = od_route.solve(stops=stops, return_directions=False)

In [21]:
result

{'checksum': 'oKu9ATnEvQE.',
 'requestID': '5d8662d2-4d71-4316-9c50-1ab22aedbd7d',
 'routes': {'fieldAliases': {'ObjectID': 'ObjectID',
   'Name': 'Name',
   'FirstStopID': 'FirstStopID',
   'LastStopID': 'LastStopID',
   'StopCount': 'StopCount',
   'Total_TravelTime': 'Total_TravelTime',
   'Total_Miles': 'Total_Miles',
   'Total_Kilometers': 'Total_Kilometers',
   'Shape_Length': 'Shape_Length'},
  'geometryType': 'esriGeometryPolyline',
  'spatialReference': {'wkid': 4326, 'latestWkid': 4326},
  'fields': [{'name': 'ObjectID',
    'type': 'esriFieldTypeOID',
    'alias': 'ObjectID'},
   {'name': 'Name',
    'type': 'esriFieldTypeString',
    'alias': 'Name',
    'length': 1024},
   {'name': 'FirstStopID',
    'type': 'esriFieldTypeInteger',
    'alias': 'FirstStopID'},
   {'name': 'LastStopID',
    'type': 'esriFieldTypeInteger',
    'alias': 'LastStopID'},
   {'name': 'StopCount', 'type': 'esriFieldTypeInteger', 'alias': 'StopCount'},
   {'name': 'Total_TravelTime',
    'type': 'e

In [ ]:
break

In [ ]:
# look for pattern with increasing travel time?
pct_esri_time.index = df_od['TravelTime']
pct_esri_time = pct_esri_time.sort_index()
pct_esri_time.plot()

In [ ]:
# Compare travel distance (m)
pct_esri_dist = ((df_od['Distance (m)'] - df_routes['Distance (m)'])/df_od['Distance (m)'])*100
#3 x-axis is index, y-axis is percent disagreement
pct_esri_dist.plot()

In [ ]:
# look for pattern with increasing distance?
pct_esri_dist.index = df_od['Distance (m)']
pct_esri_dist = pct_esri_dist.sort_index()
pct_esri_dist.plot()

In [ ]:
#df_od

In [ ]:
#df_routes